In [32]:
import os
import numpy as np #to efficiently compute functions
import pandas as pd #to work with data frames
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import zipfile #to open zipfile
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 60)
from scipy.stats import mode

# data partition
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

#filter methods
# spearman 
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

# embedded methods
from sklearn.linear_model import LassoCV

#Imputing missing values
from sklearn.impute import SimpleImputer

# encoding and scaling
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler

# model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# models
from sklearn.ensemble import RandomForestClassifier

# model assessment
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [33]:
import pickle

# Load the processed datasets

In [34]:
# Define the absolute path for saving files
processed_dir = "C:/Users/alexg/Ambiente de Trabalho/Mestrado Data Science NOVA IMS/1st semester/Machine Learning/Project_github/ML_Project_Group52/project/data/processed"

In [35]:
train_file_path = os.path.join(processed_dir, 'prepared_claims_data_train.csv')
val_file_path = os.path.join(processed_dir, 'prepared_claims_data_val.csv')
test_file_path = os.path.join(processed_dir, 'prepared_claims_data_test.csv')
test_unseen_data_path = os.path.join(processed_dir, 'prepared_claims_data_test_unseen.csv')

In [36]:
y_train_file_path = os.path.join(processed_dir, 'y_train_encoded.csv')
y_val_file_path = os.path.join(processed_dir, 'y_val_encoded.csv')
y_test_file_path = os.path.join(processed_dir, 'y_test_encoded.csv')

In [37]:
X_train = pd.read_csv(train_file_path, index_col= 0)
X_val = pd.read_csv(val_file_path , index_col= 0)
X_test = pd.read_csv(test_file_path, index_col= 0)

test_df = pd.read_csv(test_unseen_data_path, index_col=0)

In [38]:
y_train = pd.read_csv(y_train_file_path , index_col=0)
y_val = pd.read_csv(y_val_file_path)
y_test = pd.read_csv(y_test_file_path)

In [39]:
X_train.shape , X_val.shape, X_test.shape, test_df.shape

((363063, 42), (91844, 42), (114806, 42), (387975, 42))

In [40]:
y_train.shape , y_val.shape, y_test.shape  

((363063, 1), (91844, 1), (114806, 1))

In [41]:
X_train.columns

Index(['Age at Injury', 'Attorney/Representative', 'Average Weekly Wage',
       'Carrier Name', 'Carrier Type', 'Claim Identifier', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'Gender',
       'Industry Code Description', 'Medical Fee Region',
       'WCIO Cause of Injury Description', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Description', 'Zip Code', 'Number of Dependents',
       'Cause Injury Category', 'Nature of Injury Category',
       'Body Part Category', 'Age_Group', 'Wage_Group', 'Carrier Name Grouped',
       'Lag_Time', 'Accident_to_C2_Lag', 'Accident Date_Year',
       'Accident Date_Month', 'Assembly Date_Month', 'Assembly Date_Day',
       'C-2 Date_Year', 'C-2 Date_Month', 'C-2 Date_Day', 'Accident_Month_Sin',
       'Accident_Month_Cos', 'Accident_Day', 'Assembly_Year',
       'Assembly_Month_Sin', 'Assembly_Month_Cos', 'Assembly_Day',
       'C-2_Month_Sin', 'C-2_Month_Cos', 'C-2_Day'],
      dtype='object')

# Set-up pipeline

In [42]:
# building a pipeline to preprocess the data
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, learning_curve, validation_curve, RandomizedSearchCV
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import pandas as pd

In [43]:
numerical_features = [
    "Age at Injury",
    "Average Weekly Wage",
    "Number of Dependents",
    "Lag_Time",
    "Accident_to_C2_Lag",
    "Accident Date_Year",
    "Assembly Date_Day",
    "C-2 Date_Year",
    "C-2 Date_Day",
    "Accident_Month_Sin",
    "Accident_Month_Cos",
    "Accident_Day",
    "Assembly_Year",
    "Assembly_Month_Sin",
    "Assembly_Month_Cos",
    "Assembly_Day",
    "C-2_Month_Sin",
    "C-2_Month_Cos",
    "C-2_Day"
]


In [44]:
onehot_features = [
    "Gender",
    "Attorney/Representative",
    "COVID-19 Indicator",
    "Medical Fee Region",
    "Accident Date_Month",
    "Assembly Date_Month"
]


In [45]:
# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [46]:
label_features = [
    "Carrier Name",
    "Carrier Type",
    "District Name",
    "Industry Code Description",
    "Zip Code",
    "Cause Injury Category",
    "Nature of Injury Category",
    "Body Part Category",
    "Carrier Name Grouped"
]


In [47]:
ordinal_features = ['Wage_Group','Age_Group']

# Encoding and Scaling Pipeline

In [48]:
X_train.columns

Index(['Age at Injury', 'Attorney/Representative', 'Average Weekly Wage',
       'Carrier Name', 'Carrier Type', 'Claim Identifier', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'Gender',
       'Industry Code Description', 'Medical Fee Region',
       'WCIO Cause of Injury Description', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Description', 'Zip Code', 'Number of Dependents',
       'Cause Injury Category', 'Nature of Injury Category',
       'Body Part Category', 'Age_Group', 'Wage_Group', 'Carrier Name Grouped',
       'Lag_Time', 'Accident_to_C2_Lag', 'Accident Date_Year',
       'Accident Date_Month', 'Assembly Date_Month', 'Assembly Date_Day',
       'C-2 Date_Year', 'C-2 Date_Month', 'C-2 Date_Day', 'Accident_Month_Sin',
       'Accident_Month_Cos', 'Accident_Day', 'Assembly_Year',
       'Assembly_Month_Sin', 'Assembly_Month_Cos', 'Assembly_Day',
       'C-2_Month_Sin', 'C-2_Month_Cos', 'C-2_Day'],
      dtype='object')

In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363063 entries, 0 to 363062
Data columns (total 42 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Age at Injury                      363063 non-null  float64
 1   Attorney/Representative            363063 non-null  object 
 2   Average Weekly Wage                363063 non-null  float64
 3   Carrier Name                       363063 non-null  object 
 4   Carrier Type                       363063 non-null  object 
 5   Claim Identifier                   363063 non-null  float64
 6   County of Injury                   363063 non-null  object 
 7   COVID-19 Indicator                 363063 non-null  object 
 8   District Name                      363063 non-null  object 
 9   Gender                             363063 non-null  object 
 10  Industry Code Description          363063 non-null  object 
 11  Medical Fee Region                 363063 no

In [50]:
# Ensure the correct data type for ordinal features
X_train[ordinal_features] = X_train[ordinal_features].astype(str)
X_val[ordinal_features] = X_val[ordinal_features].astype(str)
X_test[ordinal_features] = X_test[ordinal_features].astype(str)

In [51]:
# Create separate transformers for each type of feature
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

onehot_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),  # Use dense matrices
    ('scaler', StandardScaler())  # Now StandardScaler will work
])

ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(
        categories=[
            ['Teen', 'Young Adult', 'Adult', 'Middle-Aged Adult', 'Older Adult', 'Senior'],
            ['Low', 'Below Average', 'Average', 'Above Average', 'High']
        ],
        handle_unknown='use_encoded_value',
        unknown_value=-1
    )),
    ('scaler', StandardScaler())  # Add scaling
])


label_transformer = Pipeline(steps=[
    ('label', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('scaler', StandardScaler())  # Add scaling
])


preprocessor = ColumnTransformer(transformers=[
    ('onehot', onehot_transformer, onehot_features),
    ('ordinal', ordinal_transformer, ordinal_features),
    ('label', label_transformer, label_features),
    ('num', numerical_transformer, numerical_features)  # Ensure numerical features are also scaled
])


In [52]:
# Fit the preprocessor only on the training data
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('onehot',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Gender', 'Attorney/Representative',
                                  'COVID-19 Indicator', 'Medical Fee Region',
                                  'Accident Date_Month',
                                  'Assembly Date_Month']),
                                ('ordinal',
                                 Pipeline(steps=[('ordinal',
                                                  OrdinalEncoder(categories=[['Teen',...
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['Age at Injury', 'Average Weekly Wage',
                                  'Number of Dependents', 'Lag_Time',
                                  'Accident_to_C2_Lag', 'Accident Date_Year',
                                  'Assembly Date_Day', 'C-2 Date_Year',
                                  'C-2 Date_Day', 'Accident_Month_Sin',
                                  'Accident_Month_Cos', 'Accident_Day',
                                  'Assembly_Year', 'Assembly_Month_Sin',
                                  'Assembly_Month_Cos', 'Assembly_Day',
                                  'C-2_Month_Sin', 'C-2_Month_Cos',
                                  'C-2_Day'])])

In [53]:
# Transform the training, validation, and test data
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed = preprocessor.transform(X_val)
X_test_transformed = preprocessor.transform(X_test)

# Select 500 observations sample

In [54]:
X_train_transformed.shape, X_val_transformed.shape, X_test_transformed.shape

((363063, 66), (91844, 66), (114806, 66))

In [55]:
# Select 500 random indices
random_indices_train = np.random.choice(X_train_transformed.shape[0], 500, replace=False)
random_indices_val = np.random.choice(X_val_transformed.shape[0], 500, replace=False)
random_indices_test = np.random.choice(X_test_transformed.shape[0], 500, replace=False)
random_indices_test_df = np.random.choice(test_df.shape[0], 500, replace=False)  # For test_df as it's a DataFrame


# Use these indices to slice the transformed datasets
X_train_transformed_sample = X_train_transformed[random_indices_train, :]
X_val_transformed_sample = X_val_transformed[random_indices_val, :]
X_test_transformed_sample = X_test_transformed[random_indices_test, :]

# Use .iloc for test_df to select rows
test_df_sample = test_df.iloc[random_indices_test_df, :]


# Print the shapes of the new transformed datasets
print(f"X_train_transformed_sample shape: {X_train_transformed_sample.shape}")
print(f"X_val_transformed_sample shape: {X_val_transformed_sample.shape}")
print(f"X_test_transformed_sample shape: {X_test_transformed_sample.shape}")
print(f"test_df_sample shape: {test_df_sample.shape}")



X_train_transformed_sample shape: (500, 66)
X_val_transformed_sample shape: (500, 66)
X_test_transformed_sample shape: (500, 66)
test_df_sample shape: (500, 42)


In [56]:
# Extract feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

In [57]:
# Assuming feature_names contains the column names
X_train_sample = pd.DataFrame(X_train_transformed, columns=feature_names)
X_val_sample = pd.DataFrame(X_val_transformed, columns=feature_names)
X_test_sample = pd.DataFrame(X_test_transformed, columns=feature_names)

test_df_sample = pd.DataFrame(test_df_sample, columns=feature_names)

In [58]:
X_train_sample.columns

Index(['onehot__Gender_F', 'onehot__Gender_M', 'onehot__Gender_Other',
       'onehot__Attorney/Representative_N',
       'onehot__Attorney/Representative_Y', 'onehot__COVID-19 Indicator_N',
       'onehot__COVID-19 Indicator_Y', 'onehot__Medical Fee Region_I',
       'onehot__Medical Fee Region_II', 'onehot__Medical Fee Region_III',
       'onehot__Medical Fee Region_IV', 'onehot__Medical Fee Region_UK',
       'onehot__Accident Date_Month_1.0', 'onehot__Accident Date_Month_2.0',
       'onehot__Accident Date_Month_3.0', 'onehot__Accident Date_Month_4.0',
       'onehot__Accident Date_Month_5.0', 'onehot__Accident Date_Month_6.0',
       'onehot__Accident Date_Month_7.0', 'onehot__Accident Date_Month_8.0',
       'onehot__Accident Date_Month_9.0', 'onehot__Accident Date_Month_10.0',
       'onehot__Accident Date_Month_11.0', 'onehot__Accident Date_Month_12.0',
       'onehot__Assembly Date_Month_1.0', 'onehot__Assembly Date_Month_2.0',
       'onehot__Assembly Date_Month_3.0', 'onehot

# Train a model to test

In [59]:
selected_features = [
    "num__Age at Injury",
    "onehot__Attorney/Representative_N",
    "num__Average Weekly Wage",
    "label__Carrier Name",  
    "num__Accident_Day",
    "onehot__Gender_M",
    "onehot__COVID-19 Indicator_N",
    "label__Zip Code",
    "label__Industry Code Description",
    "label__Nature of Injury Category",
    "label__Body Part Category"
]


In [80]:
X_train_sample['num__Age at Injury'].value_counts()

num__Age at Injury
-0.841253    8958
-0.914346    8887
-0.768161    8871
-0.987439    8785
 0.766789    8589
 0.839881    8530
 0.620603    8529
-0.695068    8528
 0.912974    8500
-0.621975    8434
 0.693696    8378
-1.060532    8340
 0.547510    8272
 0.986067    8253
-0.548882    8249
-0.402696    8248
-1.133625    8185
 1.059160    8095
-0.475789    8083
 0.474417    7950
 1.132253    7935
-0.329604    7875
-1.206717    7823
-0.256511    7762
-0.110325    7750
-0.183418    7648
-0.037232    7575
-1.279810    7439
 1.205345    7438
 0.328232    7423
 0.401324    7408
 0.035860    7323
 0.255139    7278
 0.182046    7244
 1.278438    7211
 0.108953    7051
-1.352903    6961
-1.425996    6822
 1.351531    6672
-1.499089    6027
 1.424624    5544
-1.572182    5310
 1.497717    4809
-1.645274    4705
 1.570810    4119
-1.718367    3835
 1.643902    3247
 1.716995    2404
-1.791460    2272
 1.790088    1933
 1.863181    1561
 1.936274    1289
 2.009366    1063
-1.864553     850
 2.082459

In [60]:
y_train.isna().sum()

Claim Injury Type    0
dtype: int64

In [61]:
X_train_selected = X_train_sample[selected_features]


# Train a random forest classifier
print("Training random forest classifier ...")
model_rf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_leaf=5, random_state=42)
model_rf.fit(X_train_selected, y_train)
print("Model training completed.")

Training random forest classifier ...
Model training completed.


In [81]:
rf_predictions = model_rf.predict(test_df_sample[selected_features])

In [64]:
selected_features

['num__Age at Injury',
 'onehot__Attorney/Representative_N',
 'num__Average Weekly Wage',
 'label__Carrier Name',
 'num__Accident_Day',
 'onehot__Gender_M',
 'onehot__COVID-19 Indicator_N',
 'label__Zip Code',
 'label__Industry Code Description',
 'label__Nature of Injury Category',
 'label__Body Part Category']

# Deployment of the trained model

In [62]:
file_name = 'trained_model.sav'
pickle.dump(model_rf , open(file_name,'wb')) # placeholder: Replace model_rf with our trained model

In [63]:
# Loading the saved model

loaded_model = pickle.load(open(file_name,'rb'))